In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [15]:
def city_scraper(city):    
    sleep(3)
    print(f"start: {city}")
    
    response = requests.get(f"https://data2.openstreetmap.hu/addrtel.php?addr={city}")    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    table = soup.find('table')

    rows = []
    for i, row in enumerate(table.find_all('tr')):
        if i > 0:
            rows.append([td.text.strip() for td in row.find_all('td')])
    
    content = []
    for i, row in enumerate(rows):    
        content.append(row[:4])
    
    df = pd.DataFrame(content, columns=['POSTCODE', 'CITY', 'STREET', 'NUMBER'])
    df = df[df['STREET'].str.len() > 1]
    df = df[df['CITY'].str.len() > 1]
    df = df[df['NUMBER'].str.len() > 1]
    df = df[df['POSTCODE'].str.len() > 1]
    
    df['ADDRESS'] = df['STREET'] + ' ' + df['NUMBER']
    df.drop(['STREET', 'NUMBER'], axis=1, inplace=True)
    
    return df

In [7]:
districts = ['I','II','III','IV','V','VI','VII','VIII','IX','X','XI','XII','XIII','XIV','XV','XVI','XVII','XVIII','XIX','XX','XXI','XXII','XXIII']
budapest = []
for i in districts:
    district = f"{i}.%20ker%C3%BClet"
    budapest.append(district)

In [11]:
cities = budapest

In [ ]:
df = pd.DataFrame()
all_df = pd.DataFrame()
for city in cities:
    df = city_scraper(city)
    all_df = pd.concat([all_df, df], ignore_index=True)
    print(f"--- Hozzáadva ---")

In [13]:
all_df

,POSTCODE,CITY,ADDRESS
0,1011,Budapest,Apor Péter utca 2-4
1,1011,Budapest,Batthyány tér 5-6
2,1011,Budapest,Batthyány tér 5-6
3,1011,Budapest,Batthyány tér 5-6
4,1011,Budapest,Bem rakpart 11
...,...,...,...
81751,1239,Budapest,Ócsai út 22
81752,1239,Budapest,Ócsai út 22
81753,1724,Budapest,Bevásárló utca 2-4
81754,1734,Budapest,Grassalkovich út 153


In [18]:
all_df

,POSTCODE,CITY,ADDRESS
0,1011,Budapest,Apor Péter utca 2-4
1,1011,Budapest,Batthyány tér 5-6
2,1011,Budapest,Batthyány tér 5-6
3,1011,Budapest,Batthyány tér 5-6
4,1011,Budapest,Bem rakpart 11
...,...,...,...
82054,7678,Abaliget,Széchenyi utca 35
82055,7678,Abaliget,Széchenyi utca 36
82056,7678,Abaliget,Széchenyi utca 37
82057,7678,Abaliget,Széchenyi utca 38


In [14]:
all_df.to_csv(r'C:\vremek\budapest_all.csv', sep=';', index=False)

In [ ]:
df = pd.read_csv(r'C:\vremek\adatok\nyers\magyar_telepulesek_turabazis.csv', delimiter=';')
df = df.loc[~df['City'].str.startswith('Budapest')]

cities = []
for city in df['City']:
    cities.append(city)

In [ ]:
all_df = pd.DataFrame()
for city in cities:
    try:
        df = city_scraper(city)
        all_df = pd.concat([all_df, df], ignore_index=True)
        print(f"--- Hozzáadva ---")
    except Exception as e:
        print("Hiba: ", e)
        continue

In [22]:
all_df

,POSTCODE,CITY,ADDRESS
0,8127,Aba,Dózsa György utca 121
1,8127,Aba,Esze Tamás utca 16
2,8127,Aba,Klapka György utca 14
3,8127,Aba,Petőfi Sándor utca 95
4,8127,Aba,Rákóczi Ferenc utca 12
...,...,...,...
290744,6792,Zsombó,Gárdonyi utca 21
290745,6792,Zsombó,Szegedi út 60
290746,3897,Zsujta,Fő utca 11
290747,3897,Zsujta,Fő utca 29


In [24]:
all_df.to_csv(r'C:\vremek\videki_utcak.csv', sep=';', index=False)